<a href="https://colab.research.google.com/github/scottieballs/weather_classifier/blob/develop/weatherClassifierCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weather Image Classifier

This python notebook contains the implementation of a convolutional neural network (CNN)
for classifiying weather images into the following six classes:

1. Sunny
1. Cloudy
1. Rainy
1. Foggy
1. Other

The dataset used is the Image2Weather dataset published by 

Wei-Ta Chu, Xiang-You Zheng, and Ding-Shiuan Ding, "Camera as Weather Sensor: Estimating Weather Information from Single Images," Journal of Visual Communications and Image Representation, vol. 46, pp. 233-249, 2017.



In [11]:
# Import modules
import os
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
import random
import math
import PIL
import sys

from PIL import Image
sys.modules['Image'] = Image

print(tf.__version__)
tf.test.gpu_device_name()

# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End"); 
#     };

#     var intervalId = setInterval(clickConnect, 60000);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();

# !pip install gputil
# import psutil
# import humanize
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

2.3.0


'/device:GPU:0'

In [12]:
# Mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

cwd_path = os.path.abspath(os.getcwd())
print(cwd_path)

zip_file = cwd_path +'/gdrive/My\ Drive/data/PreparedWeatherImages.zip'
if (os.path.isdir('/tmp/data') == False):
  os.mkdir('/tmp/data')
!cp $zip_file '/tmp/data'

# Unzip the dataset locally
local_zip = '/tmp/data' + '/PreparedWeatherImages.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/data')
zip_ref.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content


In [13]:
# Set the training and validation directories
train_dir = '/tmp/data/WeatherImages'
validation_dir = '/tmp/data/ValidationWeatherImages'

# Set plot_vars = True if you want to see pictures of the data 
plot_vars = False

In [14]:
if plot_vars == True:
  # Plot some of the images in the data set
  %matplotlib inline

  nrows = 1
  ncols = 6

  fig = plt.gcf()
  fig.set_size_inches(ncols * 4, nrows * 4)

  image_list = []
  for d in os.listdir(train_dir):
      train_subdir = train_dir + '/' + d
      filename = os.listdir(train_subdir)[0]
      image_list.append(os.path.join(train_subdir, filename))

  for img_path in image_list:
      sp = plt.subplot(nrows, ncols, image_list.index(img_path)+1)
      sp.axis('Off')
      
      img = mpimg.imread(img_path)
      plt.imshow(img)

  plt.show()


In [15]:
# Define callback class to stop training at 95 percent accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

checkpoint_path = cwd_path + '/gdrive/My Drive/data/training/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])




In [16]:
use_transfer_learning = True

# Transfer learning model using Inception
pre_trained_model = InceptionV3(input_shape = (200, 200, 3), 
                                include_top = False, 
                                weights = "imagenet")

for layer in pre_trained_model.layers:
  layer.trainable = False

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                  
# Add a final softmax layer for multiclass classification
x = tf.keras.layers.Dense(6, activation='softmax')(x)           

transfer_learning_model = tf.keras.Model( pre_trained_model.input, x) 

transfer_learning_model.summary()

last layer output shape:  (None, 10, 10, 768)
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d_663 (Conv2D)             (None, 99, 99, 32)   864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_663 (BatchN (None, 99, 99, 32)   96          conv2d_663[0][0]                 
__________________________________________________________________________________________________
activation_658 (Activation)     (None, 99, 99, 32)   0           batch_normalization_663[0][0]    
_________________________________________

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 198, 198, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 196, 196, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 96, 64)        2

In [17]:
# Compile the model
if use_transfer_learning:
  transfer_learning_model.compile(loss='categorical_crossentropy',
                                  optimizer=Adam(lr=0.001),
                                  metrics=['accuracy'])
  print("Using inception model")
else:
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=0.001),
                metrics=['accuracy'])
  print("Using basic created model")

Using inception model


In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.05,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (200, 200),
    batch_size = 128,
    class_mode = 'categorical')

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(200, 200),
        batch_size=32,
        class_mode='categorical')

Found 147041 images belonging to 6 classes.
Found 36757 images belonging to 6 classes.


In [20]:
if use_transfer_learning:
  history = transfer_learning_model.fit(train_generator,
                    steps_per_epoch = 100,
                    epochs = 20,
                    validation_data=validation_generator,
                    verbose = 1,
                    callbacks = [callbacks, cp_callback, lr_schedule])
else:
  history = model.fit(train_generator,
                    steps_per_epoch = 100,
                    epochs = 20,
                    validation_data=validation_generator,
                    verbose = 1,
                    callbacks = [callbacks, cp_callback, lr_schedule])

Epoch 1/20
100/100 [==============================] - ETA: 0s - loss: 3.7913 - accuracy: 0.5838
Epoch 00001: saving model to /content/gdrive/My Drive/data/training/cp.ckpt
100/100 [==============================] - 388s 4s/step - loss: 3.7913 - accuracy: 0.5838 - val_loss: 0.7766 - val_accuracy: 0.6973
Epoch 2/20
100/100 [==============================] - ETA: 0s - loss: 0.7683 - accuracy: 0.6974
Epoch 00002: saving model to /content/gdrive/My Drive/data/training/cp.ckpt
100/100 [==============================] - 369s 4s/step - loss: 0.7683 - accuracy: 0.6974 - val_loss: 0.7346 - val_accuracy: 0.7159
Epoch 3/20
100/100 [==============================] - ETA: 0s - loss: 0.7450 - accuracy: 0.7111
Epoch 00003: saving model to /content/gdrive/My Drive/data/training/cp.ckpt
100/100 [==============================] - 367s 4s/step - loss: 0.7450 - accuracy: 0.7111 - val_loss: 0.7058 - val_accuracy: 0.7257
Epoch 4/20
100/100 [==============================] - ETA: 0s - loss: 0.7288 - accuracy:

In [21]:

model.save(cwd_path + '/gdrive/My Drive/Colab Notebooks/WeatherClassifier_inception_v31-08-2020.h5') 

In [24]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss' )
plt.plot( epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')

plt.show()

AttributeError: ignored